# CSC 418 Group D Data Science Project

Category : **Major**
Group Members
 
1. P15/5620/2019 : Njagi Baraka Fadhili
2. P15/1636/2019 : Kabiru Sharleen Njeri
3. P15/1635/2019 : Obora Melanie Fayne
4. P15/137631/2019 : Ali Amina Abdi
5. P15/130607/2018 : Munyao Mary June

## Importing libraries
_____

In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
%matplotlib inline

import sys
!{sys.executable} -m pip install xgboost
import sys
!{sys.executable} -m pip install lightgbm;

from tqdm import tqdm
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from scipy.special import erfc
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score,accuracy_score,classification_report,confusion_matrix ,recall_score, precision_score
from sklearn.metrics import roc_curve, roc_auc_score

## Reading Files

____

In [7]:
train_data = pd.read_csv("./data/train.csv")
# Preview the first five rows of the train dataset
print(f'The shape of the dataset is: {train_data.shape}')
train_data.head()

The shape of the dataset is: (4459, 4993)


,ID,target,48df886f9,0deb4b6a8,34b15f335,a8cb14b00,2f0771a37,30347e683,d08d1fbe3,6ee66e115,...,3ecc09859,9281abeea,8675bec0b,3a13ed79a,f677d4d13,71b203550,137efaa80,fb36b89d9,7e293fbaf,9fc776466
0,000d6aaf2,38000000.0,0.0,0,0.0,0,0,0,0,0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
1,000fbd867,600000.0,0.0,0,0.0,0,0,0,0,0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
2,0027d6b71,10000000.0,0.0,0,0.0,0,0,0,0,0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
3,0028cbf45,2000000.0,0.0,0,0.0,0,0,0,0,0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
4,002a68644,14400000.0,0.0,0,0.0,0,0,0,0,0,...,0.0,0.0,0.0,0,0,0,0,0,0,0


In [8]:
test_data = pd.read_csv("./data/test.csv")
# Preview the first five rows of the test dataset
print(f'The shape of the dataset is: {test_data.shape}')
test_data.head()

The shape of the dataset is: (49342, 4992)


,ID,48df886f9,0deb4b6a8,34b15f335,a8cb14b00,2f0771a37,30347e683,d08d1fbe3,6ee66e115,20aa07010,...,3ecc09859,9281abeea,8675bec0b,3a13ed79a,f677d4d13,71b203550,137efaa80,fb36b89d9,7e293fbaf,9fc776466
0,000137c73,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,00021489f,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0004d7953,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,00056a333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,00056d8eb,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Data Understanding and Preparation
_____

**Things to check for**:

1. Check for fake data in the train and test datasets: *Criteria to use: It is fake if it has no unique values*
2. Check for null and duplicate values 
3. Check for Outliers
4. Check for Feature Distribution
5. Check for Feature Importance: using corellation

### Checking for Data Uniqueness

In [9]:
# let's first create a copy of the train and test data 
train_df = train_data.copy()
test_df = test_data.copy()

In [11]:
# we then drop the target and ID code columns for the train and test 
train_df.drop(train_df[['ID', 'target']], axis=1, inplace=True)
test_df.drop(test_df[['ID']], axis=1, inplace= True)

#### Checking the Train Dataset

In [ ]:
orig = [f'var_{i}' for i in range(200)]
has_one = [f'var_{i}_has_one' for i in range(200)]
has_zero = [f'var_{i}_has_zero' for i in range(200)]
not_u = [f'var_{i}_not_unique' for i in range(200)]

for f in tqdm(orig):
    unique_v = train_df[f].value_counts()
    unique_v = unique_v.index[unique_v == 1]
    train_df[f + '_u'] = train_df[f].isin(unique_v)

The cell above defines four lists of strings, orig, has_one, has_zero, and not_u, which contain the names of 200 variables in the format 'var_i' where i is a number from 0 to 199.

Next, the code enters a loop that iterates over the orig list. For each iteration of the loop, the code calculates the number of unique values in the corresponding column of the train dataframe. It then selects the values that occur only once, and creates a new column in the train dataFrame that indicates whether each value in the original column is in the list of unique values.

The tqdm function is used to wrap the loop and display a progress bar that shows the progress of the loop. This can be useful when the loop is running over a large number of iterations and you want to have some sense of how long it will take to complete.

In [ ]:
train_df['has_unique'] = train_df[[f + '_u' for f in orig]].any(axis=1)
print(train_df['has_unique'].sum())

The cell above creates a new column called 'has_unique' in the train dataframe. The new column is a boolean column that indicates whether any of the original 4992 variables in train dataset have any unique values.

The new column is created by selecting a sub-DataFrame from train data that contains all of the columns with names ending in '_u'. This sub-DataFrame is created using a list comprehension that generates a list of column names by adding '_u' to each element of the orig list. The any function is then used to check whether any of the values in the sub-DataFrame are True. The axis=1 parameter specifies that the operation should be performed column-wise, rather than row-wise.

Finally, the cell prints the sum of the 'has_unique' column, which is the number of rows in test_data where at least one of the original 4992 variables has a unique value

In [ ]:
real_train = train_df.loc[train_df['has_unique'], orig]
new_train = pd.concat([train_data[['ID', 'target']], real_train], axis=1)
print(new_train.shape)
new_train.head()

#### Checking the Test Dataset

In [ ]:
orig = [f'var_{i}' for i in range(200)]
has_one = [f'var_{i}_has_one' for i in range(200)]
has_zero = [f'var_{i}_has_zero' for i in range(200)]
not_u = [f'var_{i}_not_unique' for i in range(200)]

for f in tqdm(orig):
    unique_v = test_df[f].value_counts()
    unique_v = unique_v.index[unique_v == 1]
    test_df[f + '_u'] = test_df[f].isin(unique_v)

In [ ]:
test_df['has_unique'] = test_df[[f + '_u' for f in orig]].any(axis=1)
print(test_df['has_unique'].sum())

In [ ]:
real_test = test_df.loc[test_df['has_unique'], orig]
real_test.shape

In [ ]:
# lets create the new test data 
new_test = pd.concat([test_data['ID'], real_test], axis=1)
new_testdata = new_test.dropna()
print(new_testdata.shape)
new_testdata.head()

### Checking for Data Types, Duplicates and Missing Values

In [ ]:
new_train.info() # check datatypes

In [ ]:
new_testdata.info() # check datatypes

In [ ]:
new_train.describe() # check datatypes

In [ ]:
# describing numerical values
new_testdata.describe().T

In [ ]:
# Categorical Values/Object Values
new_train.describe(include="O").T

In [ ]:
# Categorical Values/Object Values
new_testdata.describe(include="O").T

In [ ]:
#missing value 
new_train.isnull().sum().sort_values(ascending=False) 

In [ ]:
new_testdata.isnull().sum().sort_values(ascending=False) 

In [ ]:
#duplicate data 
new_train.duplicated().sum()

In [ ]:
#duplicate data 
new_test.duplicated().sum()

**Observations**

*   We are provided with an anonymized dataset containing numeric feature variables, the numeric target column, and a string ID column
*   The dataset has 202 unique Columns and 300,000 rows
*   The dataset has 0 missing value 
*   The dataset has 0 duplicate rows 

### Checking for Outliers

#### Using Chauvenet's criterion to check for outliers

In [ ]:
def chauvenet(array):
    mean = array.mean()           # Mean of incoming array
    stdv = array.std()            # Standard deviation
    N = len(array)                # Lenght of incoming array
    criterion = 1.0/(2*N)         # Chauvenet's criterion
    d = abs(array-mean)/stdv      # Distance of a value to mean in stdv's
    prob = erfc(d)                # Area normal dist.    
    return prob < criterion       # Use boolean array outside this function

In [ ]:
numerical_features=new_train.columns[2:]

In [ ]:
outliers = dict() 
for col in [col for col in numerical_features]:
    outliers[col] = new_train[chauvenet(new_train[col].values)].shape[0]
outliers = pd.Series(outliers)

outliers.sort_values().plot(figsize=(14, 40), kind='barh').set_xlabel('Number of outliers');

In [ ]:
print('Total number of outliers in the train data: {} ({:.2f}%)'.format(sum(outliers.values), (sum(outliers.values) / new_train.shape[0]) * 100))

In [ ]:
numerical_features=new_testdata.columns[2:]

In [ ]:
outliers = dict() 
for col in [col for col in numerical_features]:
    outliers[col] = new_testdata[chauvenet(new_testdata[col].values)].shape[0]
outliers = pd.Series(outliers)

outliers.sort_values().plot(figsize=(14, 40), kind='barh').set_xlabel('Number of outliers');

In [ ]:
print('Total number of outliers in the test dataset: {} ({:.2f}%)'.format(sum(outliers.values), (sum(outliers.values) / new_testdata.shape[0]) * 100))

#### Using Z-score to check for outliers

In [ ]:
# ----------- Using z-score
#train data
data=new_train[new_train.columns.difference(['target','ID'])]
stats.zscore(data)
stats.zscore(data, axis=1)
data.apply(stats.zscore)

In [ ]:
# ----------- Using z-score

data=new_testdata[new_testdata.columns.difference(['ID'])]
stats.zscore(data)
stats.zscore(data, axis=1)
data.apply(stats.zscore)

#### Using IQR to remove outliers

In [ ]:
new_train.columns

In [ ]:
data=new_train

In [ ]:
#checking for outlier using IQR 
for i in new_train[new_train.columns.difference(['target', 'ID','has_unique'], sort=False)].columns:
  q25 = data[i].quantile(0.25)
  q75 = data[i].quantile(0.75)
  # Calculating the IQR
  IQR = q75 - q25
  # Specifying the lower and upper limit
  lower_limit = q25 - 1.5 * IQR
  upper_limit = q75 + 1.5 * IQR
  # Removing outliers
  dataset = data.loc[(data[i] > lower_limit) & (data[i] < upper_limit)]
  sns.boxplot(data = data, y = i)
  plt.show()

In [ ]:
data=new_testdata

In [ ]:
#checking for outlier using IQR in test data 
for i in new_testdata[new_testdata.columns.difference(['ID_code'], sort=False)].columns:
  q25 = data[i].quantile(0.25)
  q75 = data[i].quantile(0.75)
  # Calculating the IQR
  IQR = q75 - q25
  # Specifying the lower and upper limit
  lower_limit = q25 - 1.5 * IQR
  upper_limit = q75 + 1.5 * IQR
  # Removing outliers
  dataset = data.loc[(data[i] > lower_limit) & (data[i] < upper_limit)]
  sns.boxplot(data = data, y = i)
  plt.show()

In [ ]:
#Dataset Description After removing the outliers
new_train.describe()

In [ ]:
new_testdata.describe()

### Checking for Feature Distribution

In [ ]:
print('Distributions of first 100 columns of the dataset')

plt.figure(figsize=(40, 200))
for i, col in enumerate(list(new_train.columns)[2:102]):
    plt.subplot(25,4, i + 1) #The first argument is the number of plots in each row and the second the number of plots per column
    plt.hist(dataset[col])
    plt.title(col)

In [ ]:
print('Distributions of the last 100 columns the dataset')

plt.figure(figsize=(40, 200))
for i, col in enumerate(list(new_train.columns)[102:]):
    plt.subplot(25, 4, i + 1)
    plt.hist(dataset[col])
    plt.title(col)

**Observation**: All the variables are normalized distributed but not scaled 

#### Performing Feature Scaling

In [ ]:
# Select the columns to scale
features=new_train.columns[2:]
# Create the scaler object
scaler = StandardScaler()

# Fit the scaler to the data
scaler.fit(new_train[features])

# Scale the data
new_train[features] = scaler.fit_transform(new_train[features])

### Checking for Feature Importance using Corelation

In [ ]:
new_train.corr(method="pearson").style.background_gradient(cmap="rocket_r")

**General Interprataion of Pearson Co-relation**
1. **Perfect**: Near ± 1.
2. **High**: ± 0.50 to ± 1
3. **Moderate**: ± 0.30 to ± 0.49
4. **Low** degree: Below + 0.2
5. **None** : 0

**Observation**: 
* The correlation coefficient has values between -1 to 1
* The correlations between the features is very low being mostly between +0.008 and -0.008. There are NO strong correlations at all between the different feature

##### Checking for the target distribution aganist the variables 

In [ ]:
# Distribution of columns per target class
print("Distribution of columns per target class")
sns.set_style('darkgrid')
plt.figure(figsize=(40,200));
for i,col in enumerate(list(new_train.columns)[2:]):
    plt.subplot(50,4,i+1 ,);
    sns.distplot(new_train[new_train['target']==0][col],hist=False,label='0',color='green');
    sns.distplot(new_train[new_train['target']==1][col],hist=False,label='1',color='red');

**Observation**: The target variable is normally distributed across all the variables 

##### Checking The Target Column distribution

In [ ]:
# target value count 
new_train["target"].value_counts()

In [ ]:
# target value count plot 
sns.countplot(new_train['target'], palette='Set2')

In [ ]:
# Target Variable Analysis
plt.figure(figsize=(12,8))
sns.distplot(dataset["target"].values, bins=50, kde=True)
plt.xlabel('Target --> ', fontsize=12)
plt.title("target distribution", fontsize=14)
plt.show()

**Observation**: The target variable is imbalanced and more biased towards 0

### Splitting the train dataset for training

In [ ]:
# Select main columns to be used in training
main_cols = new_train.columns.difference(['ID_code', 'target'])
X = new_train[main_cols]
y = new_train.target

In [ ]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=42)

print(X_train.shape)
print(X_test.shape)

### Balancing the target Column

In [ ]:
# Checking Distribution of Target Variable
y_train.value_counts()

Synthetic Minority Over-sampling Technique (SMOTE) is a popular over-sampling method. it generates synthetic samples of the minority class, instead of just duplicating existing ones.

In [ ]:
smote = SMOTE()
# Fit and transform the data
X_train, y_train = smote.fit_resample(X_train, y_train)

In [ ]:
# Checking Distribution of Target Variable after normalisation
np.unique(y_train, return_counts=True)

In [ ]:
#target value count plot after normalising
sns.countplot(y_train, palette='Set2')

## Testing Different Classifier Algorithms

### 1. Logistic Regression

In [ ]:
model_lr = LogisticRegression()
model_lr.fit(X_train, y_train)

In [ ]:
# Prediction
y_pred_lr = model_lr.predict(X_test)

In [ ]:
print('Accuracy is : ', accuracy_score(y_test, y_pred_lr))
print(f'F1 score on the X_test is: {f1_score(y_test, y_pred_lr)}')
print(' recall:', recall_score(y_test, y_pred_lr))
print(' precision:',precision_score(y_test, y_pred_lr))
print('Area under the ROC curve:' , roc_auc_score(y_test, y_pred_lr))
confusion = confusion_matrix(y_test, y_pred_lr)
print(f'Confusion Matrix on the X_test is:\n {confusion}')

**F1 Score Board**

1.   Logistic Regression 

### 2. LGBM Classifier

In [ ]:
model_lgbm = LGBMClassifier()
model_lgbm.fit(X_train, y_train)

# Make predictions
y_pred_lgbm = model_lgbm.predict(X_test)

In [ ]:
print('Accuracy is : ', accuracy_score(y_test, y_pred_lgbm))
print(f'F1 score on the X_test is: {f1_score(y_test, y_pred_lgbm)}')
print(' recall:', recall_score(y_test, y_pred_lgbm))
print(' precision:',precision_score(y_test, y_pred_lgbm))
print('Area under the ROC curve:' , roc_auc_score(y_test, y_pred_lgbm))
confusion = confusion_matrix(y_test, y_pred_lgbm)
print(f'Confusion Matrix on the X_test is:\n {confusion}')

**F1 Scrore Board**

1.   LGBMClassifier : 
2.   Logistic Regression : 

Improved by ****

### 3. Random Forest Classifier

In [ ]:
model_rf = RandomForestClassifier(criterion='entropy')   
model_rf.fit(X_train,y_train)

In [ ]:
y_pred_rf = model_rf.predict(X_test)

In [ ]:
# Check accuracy, F1 score and Confusion Matrix

print('Accuracy is : ', accuracy_score(y_test, y_pred_rf))
print(f'F1 score on the X_test is: {f1_score(y_test, y_pred_rf)}')
print(' recall:', recall_score(y_test, y_pred_rf))
print(' precision:',precision_score(y_test, y_pred_rf))
print('Area under the ROC curve:' , roc_auc_score(y_test, y_pred_rf))
confusion = confusion_matrix(y_test, y_pred_rf)
print(f'Confusion Matrix on the X_test is:\n {confusion}')

**F1 Scrore Board**

1.   RandomForestClassifier :
2.   LGBMClassifier :
3.   Logistic Regression :

Improved by ****

### 4. Gradient Classifier

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

model_gb = GradientBoostingClassifier(
    n_estimators = 400,
    learning_rate = 1.0,
    min_samples_leaf = 10,
    subsample = 1.0,
)

In [ ]:
model_gb.fit(X_train, y_train)

y_pred_gb = model_gb.predict(X_test)

In [ ]:
# Check accuracy, F1 score and Confusion Matrix

print('Accuracy is : ', accuracy_score(y_test, y_pred_gb))
print(f'F1 score on the X_test is: {f1_score(y_test, y_pred_gb)}')
print(' recall:', recall_score(y_test, y_pred_gb))
print(' precision:',precision_score(y_test, y_pred_gb))
print('Area under the ROC curve:' , roc_auc_score(y_test, y_pred_gb))
confusion = confusion_matrix(y_test, y_pred_gb)
print(f'Confusion Matrix on the X_test is:\n {confusion}')

**F1 Scrore Board**

1.   Gradient Classifier :
2.   RandomForestClassifier :
3.   LGBMClassifier :
4.   Logistic Regression :

Improved by ****

#### 5. XGBoost Classifier

In [ ]:
from xgboost import XGBClassifier
model_xg = XGBClassifier()

model_xg.fit(X_train, y_train)

y_pred_xg = model_xg.predict(X_test)

print('Accuracy is : ', accuracy_score(y_test, y_pred_xg))
print(f'F1 score on the X_test is: {f1_score(y_test, y_pred_xg)}')
print(' recall:', recall_score(y_test, y_pred_xg))
print(' precision:',precision_score(y_test, y_pred_xg))
print('Area under the ROC curve:' , roc_auc_score(y_test, y_pred_xg))
confusion = confusion_matrix(y_test, y_pred_xg)
print(f'Confusion Matrix on the X_test is:\n {confusion}')

**F1 Scrore Board**

1.   Gradient Classifier :
2.   RandomForestClassifier :
3.   LGBMClassifier :
4.   Logistic Regression :
5.   XGBoost Classifier :

Improved by ****

#### 6. Ensemble Classifier

In [ ]:
# importing voting classifier
from sklearn.metrics import log_loss
from sklearn.ensemble import VotingClassifier

# Making the final model using voting classifier
final_model = VotingClassifier(
	estimators=[('lr', model_lr), ('lgbm', model_lgbm)], voting='soft')

# training all the model on the train dataset
final_model.fit(X_train, y_train)

# predicting the output on the test dataset
pred_final = final_model.predict(X_test)

# printing log loss between actual and predicted value
print(log_loss(y_test, pred_final))

print('Accuracy is : ', accuracy_score(y_test, pred_final))
print(f'F1 score on the X_test is: {f1_score(y_test, pred_final)}')
print(' recall:', recall_score(y_test, pred_final))
print(' precision:',precision_score(y_test, pred_final))
print('Area under the ROC curve:' , roc_auc_score(y_test, pred_final))
confusion = confusion_matrix(y_test, pred_final)
print(f'Confusion Matrix on the X_test is:\n {confusion}')

**F1 Scrore Board**

1.   Gradient Classifier :
2.   RandomForestClassifier :
3.   LGBMClassifier :
4.   Logistic Regression :
5.   XGBoost Classifier :
6.   Ensemble :

Improved by ****

### Predicting The Test Dataset with our Star Model

In [ ]:
# Make prediction on the test set
test_df = test_df[main_cols]
predictions = model_lgbm.predict(test_df)

# Create a submission file
sub_file = sample_sub.copy()
sub_file.predictions = predictions

# Check the distribution of our predictions
sns.countplot(sub_file.predictions)